In [39]:
import numpy as np
from qiskit import Aer
from qiskit.compiler import assemble
from qiskit.ignis.verification.tomography import GatesetTomographyFitter
from qiskit.ignis.verification.tomography import gateset_tomography_circuits
from qiskit.ignis.verification.tomography.basis import default_gateset_basis

from qiskit.providers.aer.noise.errors import QuantumError
from qiskit.providers.aer.noise.errors import depolarizing_error
from qiskit.providers.aer.noise import NoiseModel

from qiskit.quantum_info import PTM, Kraus, Choi
from qiskit.quantum_info import state_fidelity
from qiskit import QuantumCircuit

In [76]:
def collect_tomography_data(shots=10000,
                            noise_model=None,
                            gateset_basis='Standard GST'):
    backend_qasm = Aer.get_backend('qasm_simulator')
    circuits = gateset_tomography_circuits(gateset_basis=gateset_basis)
    qobj = assemble(circuits, shots=shots)
    result = backend_qasm.run(qobj, noise_model=noise_model).result()
    fitter = GatesetTomographyFitter(result, circuits, gateset_basis)
    return fitter

def gate_ptm_matrix(gate_func):
    c = QuantumCircuit(1)
    gate_func(c,c.qubits[0])
    return PTM(c).data
    
def gate_set_tomography(gate_func, noise_model=None):
    basis = default_gateset_basis()
    test_gate_label = 'TEST_GATE'
    basis.add_gate(test_gate_label, gate_func, gate_ptm_matrix(gate_func))
    fitter = collect_tomography_data(shots=10000, noise_model=noise_model, gateset_basis=basis)
    result_gates = fitter.fit()
    result_gate = result_gates[test_gate_label]
    return result_gate

def print_results(gate, expected_gate):
    print("Expected result:")
    print(np.real(np.round(expected_gate,decimals=2)))
    print()
    print("Result:")
    print(np.real(np.round(gate,decimals=2)))
    print()
    distance = np.linalg.norm(np.array(expected_gate.data) - np.array(gate.data))
    print("Distance = ", distance)

### H gate, no noise

In [81]:
gate_func = lambda circuit, qubit: circuit.h(qubit)
gate = gate_set_tomography(gate_func)
expected_gate = gate_ptm_matrix(gate_func)
print_results(gate, expected_gate)

Expected result:
[[ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0. -1.  0.]
 [ 0.  1.  0.  0.]]

Result:
[[ 1.    0.   -0.02 -0.  ]
 [ 0.01  0.    0.02 -0.99]
 [ 0.02  0.01 -0.99 -0.01]
 [-0.   -0.99 -0.02  0.02]]

Distance =  2.8175405686933286


### X gate, no noise

In [82]:
gate_func = lambda circuit, qubit: circuit.x(qubit)
gate = gate_set_tomography(gate_func)
expected_gate = gate_ptm_matrix(gate_func)
print_results(gate, expected_gate)

Expected result:
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0. -1.]]

Result:
[[ 1.02  0.01  0.01 -0.  ]
 [ 0.01  1.   -0.    0.  ]
 [-0.   -0.   -1.01 -0.  ]
 [-0.    0.   -0.01 -1.  ]]

Distance =  0.027007259473328316


### X gate, amplitude damping ($\gamma = 0.1$)

In [92]:
gamma = 0.1
A0 = [[1, 0], [0, np.sqrt(1 - gamma)]]
A1 = [[0, np.sqrt(gamma)], [0, 0]]
noise_ptm = PTM(Kraus([A0, A1])).data
noise = QuantumError([([{'name': 'kraus',
                         'qubits': [0],
                         'params': [A0, A1]}], 1)])
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(noise, ['u1', 'u2', 'u3', 'x'])

gate_func = lambda circuit, qubit: circuit.x(qubit)
gate = gate_set_tomography(gate_func,  noise_model)
expected_gate = noise_ptm @ gate_ptm_matrix(gate_func)
print_results(gate, expected_gate)

Expected result:
[[ 1.    0.    0.   -0.  ]
 [ 0.    0.95  0.    0.  ]
 [ 0.    0.   -0.95  0.  ]
 [ 0.1   0.    0.   -0.9 ]]

Result:
[[ 1.01 -0.01 -0.03  0.  ]
 [-0.01  0.94  0.02 -0.01]
 [-0.   -0.02 -0.92 -0.  ]
 [ 0.01 -0.01  0.02 -0.9 ]]

Distance =  0.10927619464570952


### X gate, depolarization ($p=0.1$)

In [97]:
p = np.sqrt(0.1)
X = [[0, p], [p, 0]]
Y = [[0, -1j*p], [1j*p, 0]]
Z = [[p, 0], [0, -p]]
noise_ptm = PTM(Kraus([X,Y,Z])).data
noise = QuantumError([([{'name': 'kraus',
                         'qubits': [0],
                         'params': [X,Y,Z]}], 1)])
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(noise, ['u1', 'u2', 'u3', 'x'])

gate_func = lambda circuit, qubit: circuit.x(qubit)
gate = gate_set_tomography(gate_func,  noise_model)
expected_gate = noise_ptm @ gate_ptm_matrix(gate_func)
print_results(gate, expected_gate)

Expected result:
[[ 0.3  0.   0.   0. ]
 [ 0.  -0.1  0.   0. ]
 [ 0.   0.   0.1  0. ]
 [ 0.   0.   0.   0.1]]

Result:
[[ 0.76 -0.15  0.03  0.27]
 [-0.05  0.41  0.37  0.02]
 [-0.06 -0.22 -0.19 -0.14]
 [-0.27  0.05  0.09 -0.08]]

Distance =  0.9874127581958578
